In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import pandas as pd
import warnings

from ipywidgets import widgets
from IPython.html.widgets import *

warnings.filterwarnings('ignore')
rcParams.update({'font.size': 15})
#plt.style.use('ggplot')
#plt.style.use('seaborn-dark-palette')
plt.style.use('fivethirtyeight')

C:\Users\Alvaro\Anaconda\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
#bear, bunny, green, squid

TASKS = 4
USERS = 100
PHASE = 'exploration'

exploration, train = 0, 1
phase = exploration

def stringToInt(arr, position, values):
    for i, v in enumerate(values):
        arr[arr[:,position] == v, position] = i
    return arr


#csvFile = '../data/test3.csv'
csvFile = '../data/monsters_data_free_only_052417.csv'
# get id(0), condition(1), phase(2), family(7), category(8), correct(11)
csv = np.genfromtxt(csvFile, dtype=np.string_, delimiter=',', usecols=(0,1,2,7,8,11), skip_header=1)
# id(0), phase(1), family(2), category(3), correct(4)
# id(0), cond(1), phase(2), family(3), category(4), correct(5)

#print(csv)

# Get monster type
monsters = [m for m in np.unique(csv[:,3])]
# Get category
#categories = [c for c in np.unique(csv[:,4])]
# Better manually, so they can be plotted easier in the order we want
categories = [b'category1D', b'categoryIgnore1D', b'category2D', b'categoryRandom']
# Get ids
ids = [i for i in np.unique(csv[:,0]) ]
# Get condition
cond = [co for co in np.unique(csv[:,1]) ]
# Get unique phases
phases = [p for p in np.unique(csv[:,2]) ]
# For converting string to int boolean
bo = [b'False', b'True']

csvInt = csv.copy()
# Convert fields to ints for easy processing
for i,j in enumerate([ids, cond, phases, monsters, categories, bo]):
    csvInt = stringToInt(csvInt, i, j)
csvInt = csvInt.astype('int')

#print(csvInt)

#[b'free', b'train']
# In free exploration there is no test section

if PHASE == 'exploration': 
    # Get only those in free exploration phase
    csvInt = csvInt[csvInt[:, 2] == phase]
    # Remove the phase column
    csvInt = np.delete(csvInt, 2, axis=1)

# id(0), cond(1), monsters(2), categories(3), correct(4)
# Split by user
splitCsv = [csvInt[csvInt[:,0]==i] for i in np.unique(csvInt[:,0])]


# Count times correct per task, times task chosen, % for both
metricsUser = []

# Store index of people that dind't explore all tasks
incomplete = []

if PHASE == 'exploration':
    cuser = 0
    # id, monster, category, success
    for user in range(np.shape(splitCsv)[0]):
        questions = splitCsv[user]
        condition = questions[0,1]
        print(questions)
        # Count number of times a task was selected
        task, ctask = np.unique(questions[:,3], return_counts=True)
        
        # Check if it explored all tasks
        if len(task) < 4:
            incomplete.append(user)
            
        else:
            print(task, ctask, ctask/np.shape(questions)[0])

            metricsTask = []
            # Count number of times the answer was correct per task and percent
            for t in range(TASKS):
                correct = np.sum(questions[questions[:,3] == t,4] == 1) 
                total = np.sum(questions[:,3] == t)
                if t in task:
                    # Task category, times task selected, % selection, #times correct on this task, % correct
                    metricsTask.append([cuser, condition, t, ctask[np.where(task==t)][0], ctask[np.where(task==t)][0]/np.shape(questions)[0], correct, np.nan_to_num(correct/total)])
                    #print(t, ctask[np.where(task==t)][0], ctask[np.where(task==t)][0]/np.shape(questions)[0], correct, np.nan_to_num(correct/total) )
                else:
                    metricsTask.append([cuser, condition, t, 0., 0., correct, np.nan_to_num(correct/total)])
                    #print(t, 0, 0, correct, np.nan_to_num(correct/total))

            metricsUser.append(metricsTask)
            cuser+=1

#print(monsters, categories)
# each element in the list contains a matrix with information for each task (of a single subject)
print(metricsUser[0])
print(metricsUser[1])
print(incomplete)


# TODO LIST OF USERS THAT DIDN'T EXPLORE ALL TASKS

[[0 0 0 2 1]
 [0 0 0 2 0]
 [0 0 0 2 1]
 ..., 
 [0 0 2 0 1]
 [0 0 2 0 1]
 [0 0 2 0 1]]
[[1 0 3 0 0]
 [1 0 3 0 1]
 [1 0 3 0 0]
 ..., 
 [1 0 3 0 0]
 [1 0 3 0 1]
 [1 0 3 0 0]]
(array([0, 1, 2, 3]), array([31, 71, 60, 88], dtype=int64), array([0, 0, 0, 0], dtype=int64))
[[2 0 3 1 0]
 [2 0 3 1 1]
 [2 0 3 1 1]
 ..., 
 [2 0 1 3 0]
 [2 0 1 3 1]
 [2 0 1 3 0]]
(array([0, 1, 2, 3]), array([156,  45,  27,  23], dtype=int64), array([0, 0, 0, 0], dtype=int64))
[[3 0 2 1 1]
 [3 0 2 1 1]
 [3 0 2 1 0]
 ..., 
 [3 0 2 1 0]
 [3 0 2 1 1]
 [3 0 2 1 0]]
(array([0, 1, 2, 3]), array([75, 92, 42, 41], dtype=int64), array([0, 0, 0, 0], dtype=int64))
[[4 0 0 0 0]
 [4 0 0 0 1]
 [4 0 0 0 0]
 ..., 
 [4 0 2 3 0]
 [4 0 2 3 1]
 [4 0 2 3 1]]
[[5 0 0 3 0]
 [5 0 0 3 0]
 [5 0 0 3 1]
 ..., 
 [5 0 0 3 0]
 [5 0 0 3 0]
 [5 0 0 3 1]]
(array([0, 1, 2, 3]), array([52, 42, 57, 99], dtype=int64), array([0, 0, 0, 0], dtype=int64))
[[6 0 3 0 1]
 [6 0 3 0 1]
 [6 0 3 0 1]
 ..., 
 [6 0 0 1 1]
 [6 0 0 1 1]
 [6 0 0 1 1]]
(array([0, 1, 2, 3

In [34]:
# participant:assignmentId,condition,interested-Bear,complex-Bear,time-Bear,progress-Bear,rule-Bear,
# future-learn-0-Bear,future-learn-1-Bear,interested-Bunny,complex-Bunny,time-Bunny,progress-Bunny,
# rule-Bunny,future-learn-0-Bunny,future-learn-1-Bunny,interested-GreenMonster,complex-GreenMonster,
# time-GreenMonster,progress-GreenMonster,rule-GreenMonster,future-learn-0-GreenMonster,
# future-learn-1-GreenMonster,interested-Squid,complex-Squid,time-Squid,progress-Squid,
# rule-Squid,future-learn-0-Squid,future-learn-1-Squid,age,gender,race,ethnicity,thoughts,comments

#bear 1:6 -> 2:8 (2:7)
#bunny 7:12 -> 9:15 (8:13)
#greenm 13:18 -> 16:22
#squid 19:24 -> 23:29

# Subjective data
csvFile = '../data/monsters_extra_data_free_only_05242017.csv'
#scsv = np.genfromtxt(csvFile, dtype=np.string_, delimiter=',', skip_header=1)#usecols=(0,5,6,9), skip_header=1)
scsv = np.genfromtxt(csvFile, dtype=np.string_, delimiter=',', skip_header=1, usecols=range(30))

# future-learn-0 (7) makes no sense for this exercise
# so remove 7, 14, 21, 28
scsv = np.delete(scsv, (7,14,21,28), axis=1)

# NOTE For now remove column future-learn-1-Bear (8), no data there
# So remove also 15, 22 and 29
#scsv = np.delete(scsv, (8,15,22,29), axis=1)

print(scsv[1])

#Reorder them according to the actual categories
print(monsters, categories)


# For those people that did not explore all tasks we still have to find a way to know what monster corresponds
# to what category so get that information from training/testing phase
csvInt2 = csv.copy()
# Convert fields to ints for easy processing
for i,j in enumerate([ids, cond, phases, monsters, categories, bo]):
    csvInt2 = stringToInt(csvInt2, i, j)
csvInt2 = csvInt2.astype('int')
# Get only those in train phase
csvInt2 = csvInt2[csvInt2[:, 2]==exploration]#train]
# For now also remove condition
csvInt2 = np.delete(csvInt2, (1,2), axis=1)
# Split by user
splitCsv = [csvInt2[csvInt2[:,0]==i] for i in np.unique(csvInt2[:,0])]



# each row consists of uid, monster, category, answers for that category
postCsv = []
cuser = 0
for user in range(np.shape(splitCsv)[0]):
    if user not in incomplete:
        # id(0), family(1), category(2), correct(3)
        # id(0), family(1), category(2), correct(3)
        # Get monster family and task category
        questions = splitCsv[user][:,1:3]

        print("=============")
        postQ = scsv[user]
        _, mIdx = np.unique(questions[:,0], return_index=True)
        _, cIdx = np.unique(questions[:,0], return_index=True)
        print(questions[np.sort(mIdx),0])
        print(questions[np.sort(cIdx),1])
        # stack monster type, complexity type
        moncat = np.vstack((questions[np.sort(mIdx),0], questions[np.sort(cIdx),1])).T
        moncat = moncat[moncat[:,0].argsort()]
        print(moncat)
        for n, mon in enumerate(moncat[:,0]):
            #Use this when ALL answer columns are used
            #answ = postQ[n*7+2:7*(n+1)+2]
            answ = postQ[n*6+2:6*(n+1)+2]
            # stack user id, monster id, category id, answers for that category
            print(np.hstack((user, mon, moncat[n,1], answ.tolist())))
            postCsv.append(np.hstack((cuser, mon, moncat[n,1], answ.tolist())).tolist())
        print(questions)
        cuser+=1
print(postCsv)

#TODO: EXCLUDE USERS THAT DIDN'T EXPLORE EVERYTHING
# CHECK THE SECTION YOU HAVE WITH EXPLORATION AND COMMENTED THE TRAIN DO YOU EVEN NEED IT IN THIS CASE?

[b'A12QNNTNQOEQJG:31QTRG6Q2USZQU35DBL8AUPGQIZYPN' b'0' b'9' b'6' b'4' b'8'
 b'8' b'6' b'2' b'9' b'2' b'2' b'2' b'4' b'8' b'9' b'7' b'5' b'6' b'6'
 b'10' b'3' b'6' b'8' b'8' b'7']
[b'Bear', b'Bunny', b'GreenMonster', b'Squid'] [b'category1D', b'categoryIgnore1D', b'category2D', b'categoryRandom']
[3 2 0 1]
[0 3 1 2]
[[0 1]
 [1 2]
 [2 3]
 [3 0]]
[b'1' b'0' b'1' b'9' b'6' b'4' b'8' b'8' b'6']
[b'1' b'1' b'2' b'2' b'9' b'2' b'2' b'2' b'4']
[b'1' b'2' b'3' b'8' b'9' b'7' b'5' b'6' b'6']
[b'1' b'3' b'0' b'10' b'3' b'6' b'8' b'8' b'7']
[[3 0]
 [3 0]
 [3 0]
 [3 0]
 [3 0]
 [3 0]
 [3 0]
 [3 0]
 [3 0]
 [2 3]
 [2 3]
 [2 3]
 [2 3]
 [2 3]
 [2 3]
 [2 3]
 [2 3]
 [2 3]
 [2 3]
 [2 3]
 [2 3]
 [2 3]
 [2 3]
 [2 3]
 [2 3]
 [2 3]
 [2 3]
 [2 3]
 [2 3]
 [2 3]
 [2 3]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 2]
 [1 2]
 [1 2]
 [1 2]
 [1

In [35]:
postCsv = np.asarray(postCsv)
print(postCsv)
print(np.shape(postCsv))

[[b'0' b'0' b'1' ..., b'8' b'8' b'6']
 [b'0' b'1' b'2' ..., b'2' b'2' b'4']
 [b'0' b'2' b'3' ..., b'5' b'6' b'6']
 ..., 
 [b'86' b'1' b'3' ..., b'4' b'3' b'7']
 [b'86' b'2' b'2' ..., b'2' b'4' b'3']
 [b'86' b'3' b'0' ..., b'2' b'2' b'3']]
(348, 9)


In [36]:
float_formatter = lambda x: "%.2f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})
np.set_printoptions(precision=2)

metricsUser = np.asarray(metricsUser, dtype='float')
NUSERS = np.shape(metricsUser)[0]
postCsv = postCsv.astype('float')

# Order by first column (id) and then by third column (category task complexity)
postCsv = postCsv[np.lexsort((postCsv[:,2], postCsv[:,0]))]

# user, condition, category task (complexity), #times task selected, % selection, #times correct on this task, % correct
#print(metricsUser)
metricsUser = metricsUser.reshape(NUSERS*TASKS,7)
#print(metricsUser)

# user, monster id, category task (complexity), answers for that category
#print(postCsv)

print(np.shape(metricsUser), np.shape(postCsv))

# Check that columns (user and task category complexity) are the same in order to fusion them
if np.array_equal(metricsUser[:,0], postCsv[:,0]) and np.array_equal(metricsUser[:,2], postCsv[:,2]):
    finalStats = np.hstack((metricsUser, postCsv[:,3:]))
    print(finalStats)
else:
    print('columns do not match, someone did not select all tasks')

(348, 7) (348, 9)
[[  0.   0.   0. ...,   8.   8.   7.]
 [  0.   0.   1. ...,   8.   8.   6.]
 [  0.   0.   2. ...,   2.   2.   4.]
 ..., 
 [ 86.   0.   1. ...,   5.   6.   6.]
 [ 86.   0.   2. ...,   2.   4.   3.]
 [ 86.   0.   3. ...,   4.   3.   7.]]


In [37]:
#np.savetxt('strategic-train.txt', finalStats)
#np.savetxt('strategic-test.txt', finalStats)

#np.savetxt('free-free.txt', metricsUser)
np.savetxt('free-free.txt', finalStats)

In [29]:
metricsUser

array([[  0.  ,   0.  ,   0.  , ...,   0.12,  18.  ,   0.58],
       [  0.  ,   0.  ,   1.  , ...,   0.28,  39.  ,   0.55],
       [  0.  ,   0.  ,   2.  , ...,   0.24,  32.  ,   0.53],
       ..., 
       [ 86.  ,   0.  ,   1.  , ...,   0.25,  27.  ,   0.42],
       [ 86.  ,   0.  ,   2.  , ...,   0.14,  19.  ,   0.53],
       [ 86.  ,   0.  ,   3.  , ...,   0.35,  40.  ,   0.45]])

In [30]:
finalStats

array([[  0.,   0.,   0., ...,   8.,   8.,   7.],
       [  0.,   0.,   1., ...,   8.,   8.,   6.],
       [  0.,   0.,   2., ...,   2.,   2.,   4.],
       ..., 
       [ 86.,   0.,   1., ...,   5.,   6.,   6.],
       [ 86.,   0.,   2., ...,   2.,   4.,   3.],
       [ 86.,   0.,   3., ...,   4.,   3.,   7.]])

In [31]:
print(np.shape(finalStats))

(348, 13)
